This notebook is the first in a series of notebooks where we will analyze this data on oil subsidies. We will be combining this data with other data from the world bank and the Economist Intelligence Unit to attempt to predict the likelihood of a country regulating the oil market in a given year. Such predictions have many potential use cases. For example, businesses which regularly produce or consume large quantities of oil may want these predictions to better anticipate future market conditions.
In this series of notebooks, we will import several different datasets, do some initial exploration of them, and combine them into one dataset. We will then analyze this dataset and use it to predict whether or not countries will regulate their oil markets, and how they will do so.

In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np

In [2]:
# importing data files that are in csv format
# Using skiprows=4 since the first 4 rows are headers, and are not part of dataset
# GDP of each country
gdp_data = pd.read_csv('../data/external/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_134819_copy.csv', skiprows=4)
# population of each country

population_data = pd.read_csv('../data/external/API_SP.POP.TOTL_DS2_en_csv_v2_130083_copy.csv', skiprows=4)

# land area of each country
land_area_data = pd.read_csv('../data/external/API_AG.LND.TOTL.K2_DS2_en_csv_v2_123023_copy.csv', skiprows=4)

# Democracy Index of each country (see bibliography for explanation of democracy index)
democracy_index_data = pd.read_csv('../data/external/EIU_DI_WIDEF.csv')

In [3]:
# importing excel document containing Akcura's data on oil market regulations
fuel_subsidy_data = pd.read_excel('../data/external/Global_Fuel_Subsidies_&_Price_Control_Measures_Database_copy.xlsx', engine='openpyxl', sheet_name=None)


We will refer to data on GDP, population, land area, and democracy indices as fundamental data, and refer to the data on oil regulations as fuel subsidy data.

In [4]:
fund_data = {'gdp data': gdp_data, 'population data': population_data, 'land area data': land_area_data , 'democracy index data': democracy_index_data}
for (name,dataset) in fund_data.items():
    print(name + "\n")
    print(dataset.shape)
    print(dataset.head())


gdp data

(266, 70)
                  Country Name Country Code                Indicator Name  \
0                        Aruba          ABW  GDP per capita (current US$)   
1  Africa Eastern and Southern          AFE  GDP per capita (current US$)   
2                  Afghanistan          AFG  GDP per capita (current US$)   
3   Africa Western and Central          AFW  GDP per capita (current US$)   
4                       Angola          AGO  GDP per capita (current US$)   

   Indicator Code        1960        1961        1962        1963        1964  \
0  NY.GDP.PCAP.CD         NaN         NaN         NaN         NaN         NaN   
1  NY.GDP.PCAP.CD  186.121835  186.941781  197.402402  225.440494  208.999748   
2  NY.GDP.PCAP.CD         NaN         NaN         NaN         NaN         NaN   
3  NY.GDP.PCAP.CD  121.939925  127.454189  133.827044  139.008291  148.549379   
4  NY.GDP.PCAP.CD         NaN         NaN         NaN         NaN         NaN   

         1965  ...          20

Looking at all the fundamental data, there are a few observations we can make. First, we can see that for the first three datasets, there are 266 rows, which is more than the number of countries in the world! However, looking at the first five rows, the explanation for this becomes clear. Some rows refer to regions, not just countries. Data on regions will not be useful for our analysis, so we will need to find a way to remove it later. We can also see that all datasets have data going at least as far back as 2006. The oil subsidy analysis only goes as far back as 2021, so this is plenty, and we will be ignoring data older than 2020. Where there may be insights that could some from looking more years is the past, that is outside the scope of this analysis. Finally, we can see that there are some missing values. If we're missing some of the values we need, we will need to do some pre-processing to deal with that.

In [5]:
for (name,dataset) in fund_data.items():
    # We are only using data from years starting in 2020
    for year in range(2020,2025,1):
        print("Number of null values in the " + name + "set in the year " + str(year) + ":")
        print(dataset[str(year)].isna().sum())

Number of null values in the gdp dataset in the year 2020:
9
Number of null values in the gdp dataset in the year 2021:
9
Number of null values in the gdp dataset in the year 2022:
10
Number of null values in the gdp dataset in the year 2023:
18
Number of null values in the gdp dataset in the year 2024:
35
Number of null values in the population dataset in the year 2020:
1
Number of null values in the population dataset in the year 2021:
1
Number of null values in the population dataset in the year 2022:
1
Number of null values in the population dataset in the year 2023:
1
Number of null values in the population dataset in the year 2024:
1
Number of null values in the land area dataset in the year 2020:
2
Number of null values in the land area dataset in the year 2021:
2
Number of null values in the land area dataset in the year 2022:
3
Number of null values in the land area dataset in the year 2023:
51
Number of null values in the land area dataset in the year 2024:
266
Number of null

With some missing values in all the datasets, we may need to address them. However, there are more rows in these datasets than in our dataset on oil subsidies, so we will only need to address missing values for countries that also exist in those datasets.

Before we can do this, we will need to combine these various datasets into 1 large dataset containing all the columns we are using for this analysis. That will be the purpose of the next few blocks of code. First, we will look at what our oil subsidy dataset looks like.

In [6]:
print(fuel_subsidy_data.keys())

dict_keys(['Title Page', 'Contents', 'Oil Net Trade Status', 'Coding', '2024', '2023', '2022', '2021', 'Detailed Descriptions', 'Fuel Price Measures&Subs. 2024', 'Fuel Price Measures&Subs. 2023', 'Fuel Price Measures&Subs. 2022', 'Fuel Price Measures&Subs. 2021'])


For this analysis, we will only be using some of this data. Specifically, we will use oil net trade status data and data from each year contained in the datasets. These sheets for each year are named 2021, 2022, 2023, and 2024.

In [7]:
print(fuel_subsidy_data['Oil Net Trade Status'].head())


  Economy & Region     Unnamed: 1                 Unnamed: 2  \
0           Economy  Economy Code  WBG Region Classification   
1          Albania            ALB                        ECA   
2          Algeria            DZA                       MENA   
3           Angola            AGO                        SSA   
4        Argentina            ARG                        LAC   

            Net Trade Status -\nOil refined products  \
0  0 - no data\n1 - net refined products importer...   
1                                                  1   
2                                                  2   
3                                                  1   
4                                                  1   

                      Net Trade Status - \nCrude oil  \
0  0 - no data\n1 - net crude importer\n2 - net c...   
1                                                  1   
2                                                  2   
3                                                  2  

This doesn't show full descriptions, but does give a general idea. This analysis is primarily concerned with fuel subsidies and price controls that exist for end user consumption, so we will only be looking at the net trade status of refined products. Hence, we will only use data on net trade status for refined products. 0 is no data, 1 is for importer, and 2 is for exporter. To make this useful for doing quantitative analysis, and machine learning, we will convert 0 to 1, since there are very few instances of it, and there are more net importers than exporters.

In [8]:
data_for_model = fuel_subsidy_data['Oil Net Trade Status'][['Economy & Region ', 'Unnamed: 1', 'Net Trade Status -\nOil refined products']].copy()
data_for_model.columns = ['Country', 'Country Code', 'Net Trade Status']
# Dropping row containing headers in favor of our headers.
data_for_model.drop([0], inplace=True)

In [9]:
for year in range(2021,2025,1):
    print(fuel_subsidy_data[str(year)].head())

  ECONOMY & REGION     Unnamed: 1                 Unnamed: 2  \
0           Economy  Economy Code  WBG Region Classification   
1          Albania            ALB                        ECA   
2          Algeria            DZA                       MENA   
3           Angola            AGO                        SSA   
4        Argentina            ARG                        LAC   

                         Unnamed: 3  \
0  Income group (WB Classification)   
1              Upper-middle Income    
2               Upper-middle Income   
3               Lower-middle Income   
4              Upper-middle Income    

                                          Unnamed: 4  \
0  Fuel Price Control\n\n0 - Deregulated Prices\n...   
1                                                  0   
2                                                  1   
3                                                  1   
4                                                  1   

                                          U

This also doesn't show full descriptions, but we can see a few things. The data for each year looks identical formatting wise, so we can treat them all the same way. We can also see the three columns which we will be trying to predict for our analysis. Fuel price control shows whether or not a country has regulated oil prices, or in other words, if they have implemented price controls. A 1 means they have, and a 0 means they haven't. The fuel subsidy gives information on if a country has implemented fuel subsidies. 0 corresponds to no subsidy, 1 means some subsidies, 2 means chronic price freezing, and 3 is unclear. Since this is not ordinal, we will use pd.get_dummies to handle this column. Finally the fuel tax reduction is self explanatory, and is another type of subsidy. 1 means it is present, and 0 means it is not. This covers all the columns we will be trying to predict. Finally, there are several columns on whether or not a country is considering implementing certain policies in the future. We will be excluding these from our analysis. This way, the models developed here can be used without knowing about the internal policy proposals in a given country.

In [10]:
for year in range(2021,2025,1):
    new_data_for_model = fuel_subsidy_data[str(year)][['Unnamed: 1', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']].copy()
    new_data_for_model.columns = ['Country Code', 'Price Control in ' + str(year), 'Subsidy in ' + str(year), 'Tax Reduction in ' + str(year)]
    new_data_for_model.drop([0], inplace=True)
    data_for_model = data_for_model.merge(new_data_for_model, how='inner', on='Country Code')

In [11]:
print(data_for_model.head())

      Country Country Code Net Trade Status Price Control in 2021  \
0    Albania           ALB                1                     0   
1    Algeria           DZA                2                     1   
2     Angola           AGO                1                     1   
3  Argentina           ARG                1                     1   
4    Armenia           ARM                1                     0   

  Subsidy in 2021 Tax Reduction in 2021 Price Control in 2022 Subsidy in 2022  \
0               1                     0                     1               1   
1               2                     0                     1               2   
2               2                     0                     1               2   
3               0                     0                     1               0   
4               0                     0                     0               0   

  Tax Reduction in 2022 Price Control in 2023 Subsidy in 2023  \
0                     0          

With the data on oil subsidies all set, it's time to add the fundamental data we briefly looked at earlier. Fundamental data will change each year, so we will use all years starting with 2020. We will look at addressing missing values when the need arises.

In [12]:
years = [str(i) for i in range(2020,2025,1)]
for name,dataset in fund_data.items():
    if name != "democracy index data":
        data_to_add = dataset[['Country Code'] + years].copy()
    else:
        data_to_add = dataset[['REF_AREA'] + years].copy()
        data_to_add.rename(columns={'REF_AREA': 'Country Code'}, inplace=True)
    data_to_add.rename(columns=lambda x: x if x == 'Country Code' else name + " in " + x, inplace=True)
    print(data_to_add.head())


  Country Code  gdp data in 2020  gdp data in 2021  gdp data in 2022  \
0          ABW      22855.932320      27200.061079      30559.533535   
1          AFE       1344.103210       1522.393346       1628.318944   
2          AFG        510.787063        356.496214        357.261153   
3          AFW       1680.039332       1765.954788       1796.668633   
4          AGO       1449.922867       1925.874661       2929.694455   

   gdp data in 2023  gdp data in 2024  
0      33984.790620               NaN  
1       1510.742951       1567.635839  
2        413.757895               NaN  
3       1599.392983       1284.154441  
4       2309.534130       2122.083690  
  Country Code  population data in 2020  population data in 2021  \
0          ABW                 108587.0                 107700.0   
1          AFE              694446100.0              713090928.0   
2          AFG               39068979.0               40000412.0   
3          AFW              474569351.0              48

These are the columns we wanted! Now it's time to join them with the oil subsidy data. We will use a left join, since we want all the rows (countries) that were analyzed in the original study, but no others.

In [13]:
years = [str(i) for i in range(2020,2025,1)]
for name,dataset in fund_data.items():
    if name != "democracy index data":
        data_to_add = dataset[['Country Code'] + years].copy()
    else:
        data_to_add = dataset[['REF_AREA'] + years].copy()
        data_to_add.rename(columns={'REF_AREA': 'Country Code'}, inplace=True)
    data_to_add.rename(columns=lambda x: x if x == 'Country Code' else name + " in " + x, inplace=True)
    data_for_model = data_for_model.merge(data_to_add, how='left', on='Country Code')
print(data_for_model.head())


      Country Country Code Net Trade Status Price Control in 2021  \
0    Albania           ALB                1                     0   
1    Algeria           DZA                2                     1   
2     Angola           AGO                1                     1   
3  Argentina           ARG                1                     1   
4    Armenia           ARM                1                     0   

  Subsidy in 2021 Tax Reduction in 2021 Price Control in 2022 Subsidy in 2022  \
0               1                     0                     1               1   
1               2                     0                     1               2   
2               2                     0                     1               2   
3               0                     0                     1               0   
4               0                     0                     0               0   

  Tax Reduction in 2022 Price Control in 2023  ... land area data in 2020  \
0                    

Now that we have everything in one dataframe, it's time to process all the columns that need to be processed. The first thing to do is look for missing values.

In [14]:
for name, column in data_for_model.items():
    if name[0:7] != 'Country':
        print(name)
        print(np.sum(column.isnull()))

Net Trade Status
0
Price Control in 2021
0
Subsidy in 2021
0
Tax Reduction in 2021
0
Price Control in 2022
0
Subsidy in 2022
0
Tax Reduction in 2022
0
Price Control in 2023
0
Subsidy in 2023
0
Tax Reduction in 2023
0
Price Control in 2024
0
Subsidy in 2024
0
Tax Reduction in 2024
0
gdp data in 2020
3
gdp data in 2021
3
gdp data in 2022
3
gdp data in 2023
4
gdp data in 2024
10
population data in 2020
2
population data in 2021
2
population data in 2022
2
population data in 2023
2
population data in 2024
2
land area data in 2020
2
land area data in 2021
2
land area data in 2022
2
land area data in 2023
2
land area data in 2024
154
democracy index data in 2020
12
democracy index data in 2021
12
democracy index data in 2022
12
democracy index data in 2023
12
democracy index data in 2024
12


Our fundamental data does have some null values, which we will need to address. However, that will be part of the next notebook, since this notebook has gotten us to a point where we have all our raw data, which has not been modified, as it has only been assembled from various datasets.

In [15]:
data_for_model.to_csv('../data/raw/assembled_data.csv')